In [5]:
import time
import numpy as np
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier as SklearnBaggingClassifier
from bagging import BaggingClassifier

In [6]:
datasets = {
        "Iris": load_iris(return_X_y=True),
        "Wine": load_wine(return_X_y=True)
    }
results = []

In [7]:
def evaluate_model_cv(model, X, y, cv=5):
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv)
    return np.mean(scores)

def train_and_measure_time(model, X, y, cv=5):
    start_time = time.time()
    model.fit(X, y)
    train_time = time.time() - start_time

    score = evaluate_model_cv(model, X, y, cv=cv)
    return train_time, score

In [8]:
for ds_name, (X, y) in datasets.items():
    print(f"\n=== Dataset: {ds_name} ===")

    my_model = BaggingClassifier(
        estimator=DecisionTreeClassifier(),
        n_estimators=10,
        accuracy_threshold=0.6,
        random_state=42,
        n_jobs=2
    )

    try:
        my_time, my_score = train_and_measure_time(my_model, X, y, cv=5)
        print(f"CustomBagging:       time={my_time:.4f} s,  accuracy(cv)={my_score:.4f}")
    except TimeoutError as e:
        my_time = None
        my_score = None
        print(f"CustomBagging:       FAILED (TimeoutError)")

    sklearn_model = SklearnBaggingClassifier(
        estimator=DecisionTreeClassifier(),
        n_estimators=10,
        random_state=42,
        n_jobs=2
    )
    sk_time, sk_score = train_and_measure_time(sklearn_model, X, y, cv=5)
    print(f"SklearnBagging:  time={sk_time:.4f} s,  accuracy(cv)={sk_score:.4f}")
    results.append((ds_name, my_time, my_score, sk_time, sk_score))


=== Dataset: Iris ===
CustomBagging:       time=5.5206 s,  accuracy(cv)=0.9133
SklearnBagging:  time=5.3044 s,  accuracy(cv)=0.9533

=== Dataset: Wine ===
CustomBagging:       time=0.0237 s,  accuracy(cv)=0.9046
SklearnBagging:  time=10.3730 s,  accuracy(cv)=0.9386
